In [3]:
import pandas as pd

In [4]:
final_new_df = pd.read_pickle('final_new_df.pkl')
final_new_df.head()

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,price,rating,review_count
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,$,3.0,29.0
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,$$,3.5,182.0
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,$$,3.5,99.0
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,$$,3.0,21.0
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,$,4.0,97.0


In [93]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3959 # m

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

In [94]:
def get_distance(data, pt):
    return distance((data['coordinates.latitude'], data['coordinates.longitude']), pt)

def count_less_than_distance(data, distances):
    df = final_new_df.apply(get_distance, axis=1, 
                                   args=((data['coordinates.latitude'], data['coordinates.longitude']),))
    counts = []
    for distance in distances:
        filtered_df = final_new_df[df < distance]
        similar = 0
        for i, row in filtered_df.iterrows():
            for category1 in data['categories']:
                alias1 = category1['alias']
                for category2 in row['categories']:
                    alias2 = category2['alias']
                    if alias1 == alias2:
                        similar += 1;            
        counts.append(len(df[df < distance]))
        counts.append(similar)
    return pd.Series(counts)


In [95]:
distance_counts = final_new_df.apply(count_less_than_distance, axis=1, args=([0.5, 1, 2, 3],))

In [96]:
distance_counts.head()

,0,1,2,3,4,5,6,7
0,12,8,18,10,55,25,112,36
1,21,9,44,13,98,23,167,35
2,5,3,30,11,80,24,136,34
3,9,5,32,10,83,16,241,30
4,3,1,5,1,5,1,5,1


In [97]:
len(distance_counts)

3699

In [98]:
final_new_df['0.5m_count'] = distance_counts.iloc[:,0]
final_new_df['0.5m_count_similar'] = distance_counts.iloc[:,1]
final_new_df['1m_count']   = distance_counts.iloc[:,2]
final_new_df['1m_count_similar']   = distance_counts.iloc[:,3]
final_new_df['2m_count']   = distance_counts.iloc[:,4]
final_new_df['2m_count_similar']   = distance_counts.iloc[:,5]
final_new_df['3m_count']   = distance_counts.iloc[:,6]
final_new_df['3m_count_similar']   = distance_counts.iloc[:,7]

In [99]:
final_new_df.head()

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,...,rating,review_count,0.5m_count,0.5m_count_similar,1m_count,1m_count_similar,2m_count,2m_count_similar,3m_count,3m_count_similar
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,...,3.0,29.0,12,8,18,10,55,25,112,36
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,...,3.5,182.0,21,9,44,13,98,23,167,35
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,...,3.5,99.0,5,3,30,11,80,24,136,34
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,...,3.0,21.0,9,5,32,10,83,16,241,30
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,...,4.0,97.0,3,1,5,1,5,1,5,1


In [122]:
final_new_df.dropna(subset=['alias'], inplace=True)

In [125]:
def check_is_fastfood(data):
    categories = data['categories']
    try:
        for cat in categories:
            if cat['alias'] == 'hotdogs':
                return True
    except TypeError:
        print(data)
    return False

final_new_df['is_fastfood'] = final_new_df.apply(check_is_fastfood, axis=1)
final_new_df.head()

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,...,review_count,0.5m_count,0.5m_count_similar,1m_count,1m_count_similar,2m_count,2m_count_similar,3m_count,3m_count_similar,is_fastfood
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,...,29.0,12,8,18,10,55,25,112,36,True
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,...,182.0,21,9,44,13,98,23,167,35,False
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,...,99.0,5,3,30,11,80,24,136,34,False
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,...,21.0,9,5,32,10,83,16,241,30,False
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,...,97.0,3,1,5,1,5,1,5,1,False


In [209]:
all_categories  = pd.read_csv("../data/yelp-all-categories.txt", squeeze=True, header=None)
all_categories = all_categories.apply(lambda x: x.split("(")[-1][:-1])

In [210]:
len(all_categories)

88

In [211]:
def get_category(data):
    categories = data['categories']
    try:
        for cat1 in categories:
            for cat2 in all_categories.iteritems():
                if cat1['alias'] == cat2[1]:
                    return cat1['alias']
    except TypeError:
        raise
    return ''

final_new_df['cuisine'] = final_new_df.apply(get_category, axis=1)
final_new_df.head()

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,...,1m_count,1m_count_similar,2m_count,2m_count_similar,3m_count,3m_count_similar,is_fastfood,cuisine,price_ordinal,income
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,...,18,10,55,25,112,36,True,hotdogs,1,53800.0
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,...,44,13,98,23,167,35,False,italian,2,84000.0
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,...,30,11,80,24,136,34,False,tradamerican,2,27835.0
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,...,32,10,83,16,241,30,False,pizza,2,42807.0
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,...,5,1,5,1,5,1,False,mexican,1,30764.0


In [138]:
nan_count=0
def convert_price(price_string):
    global nan_count
    try:
        return {
            '$'     : 1,
            '$$'    : 2,
            '$$$'   : 3,
            '$$$$'  : 4,
        }[price_string]
    except KeyError:
        nan_count += 1
    return 0

final_new_df['price_ordinal'] = final_new_df['price'].apply(convert_price)
print(nan_count)
final_new_df.head()

105


,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,...,0.5m_count_similar,1m_count,1m_count_similar,2m_count,2m_count_similar,3m_count,3m_count_similar,is_fastfood,cuisine,price_ordinal
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,...,8,18,10,55,25,112,36,True,sandwiches,1
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,...,9,44,13,98,23,167,35,False,italian,2
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,...,3,30,11,80,24,136,34,False,burgers,2
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,...,5,32,10,83,16,241,30,False,pizza,2
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,...,1,5,1,5,1,5,1,False,mexican,1


In [141]:
final_new_df.loc[final_new_df['price_ordinal'] == 0][['alias', 'name', 'is_closed', 'price', 'price_ordinal']]

,alias,name,is_closed,price,price_ordinal
84,amc-dine-in-theatres-esplanade-14-phoenix,AMC Dine-in Theatres Esplanade 14,False,NaN,0
95,gilbert-primary-care-mesa,Gilbert Primary Care,False,NaN,0
119,stacys-off-da-hook-bbq-and-soul-food-phoenix,Stacy's Off Da Hook BBQ and Soul Food,False,NaN,0
139,indigo-joes-gilbert,Indigo Joe's,True,NaN,0
147,subway-surprise,Subway,False,NaN,0
152,turf-paradise-phoenix,Turf Paradise,False,NaN,0
158,chi-chis-tacos-phoenix,Chi Chi's Tacos,True,NaN,0
182,su-clinica-del-valle-mesa,Su Clinica Del Valle,False,NaN,0
294,papa-johns-pizza-phoenix-5,Papa John's Pizza,False,NaN,0
327,speckled-hen-apache-junction,Speckled Hen,True,NaN,0


In [155]:
final_new_df_income = pd.read_pickle('final_new_df_income.pkl')

In [156]:
df = final_new_df.merge(final_new_df_income[['alias', 'income']], on=['alias'], how='inner')
print(len(df), len(final_new_df), len(final_new_df_income))
df.head()

3709 3543 3543


,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,...,1m_count,1m_count_similar,2m_count,2m_count_similar,3m_count,3m_count_similar,is_fastfood,cuisine,price_ordinal,income
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,...,18,10,55,25,112,36,True,sandwiches,1,53800.0
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,...,44,13,98,23,167,35,False,italian,2,84000.0
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,...,30,11,80,24,136,34,False,burgers,2,27835.0
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,...,32,10,83,16,241,30,False,pizza,2,42807.0
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,...,5,1,5,1,5,1,False,mexican,1,30764.0


In [157]:
print(len(df))
df.drop_duplicates(subset=['alias'], inplace=True)
print(len(df))

3709
3460


In [158]:
final_new_df = df

In [5]:
def check_is_chain(name):
    size = len(final_new_df[final_new_df['name'] == name])
    return size > 1

final_new_df['is_chain'] = final_new_df['name'].apply(check_is_chain)
final_new_df.head()


,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,price,rating,review_count,is_chain
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,$,3.0,29.0,True
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,$$,3.5,182.0,True
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,$$,3.5,99.0,True
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,$$,3.0,21.0,True
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,$,4.0,97.0,False


In [6]:
final_new_df.to_pickle('is_chain_feature.pkl')

In [ ]:
final_new_df.to_pickle('final_new_df_features.pkl')

In [189]:
final_new_df.columns

Index(['alias', 'categories', 'coordinates.latitude', 'coordinates.longitude',
       'hours', 'id', 'is_closed', 'location.display_address',
       'location.zip_code', 'name', 'price', 'rating', 'review_count',
       '0.5m_count', '0.5m_count_similar', '1m_count', '1m_count_similar',
       '2m_count', '2m_count_similar', '3m_count', '3m_count_similar',
       'is_fastfood', 'cuisine', 'price_ordinal', 'income'],
      dtype='object')

In [203]:
final_new_df['cuisine'].unique()

array(['hotdogs', 'italian', 'tradamerican', '', 'mexican', 'british',
       'japanese', 'tex-mex', 'chinese', 'cajun', 'vegetarian',
       'newamerican', 'persian', 'thai', 'hawaiian', 'asianfusion',
       'greek', 'cafes', 'southern', 'vietnamese', 'mediterranean',
       'indpak', 'caribbean', 'cuban', 'mongolian', 'latin', 'vegan',
       'taiwanese', 'korean', 'creperies', 'french', 'peruvian', 'irish',
       'mideastern', 'gastropubs', 'comfortfood', 'filipino', 'ethiopian',
       'modern_european', 'spanish', 'newmexican', 'pakistani', 'german',
       'african', 'turkish', 'polish', 'cambodian', 'scandinavian',
       'russian', 'afghani', 'belgian'], dtype=object)

In [212]:
len(final_new_df['cuisine'])

3460

In [213]:
len(final_new_df.loc[final_new_df['cuisine'] == ''])

638

In [214]:
# final_new_df.to_csv("cusine_info.csv")